# Importer les packages

In [103]:
import pandas as pd
import random
import nltk
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer


import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\yanth\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\yanth\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\yanth\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\yanth\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Charger les données

La dernière colonne `Bug` est créée de manière aléatoire : elle prend la valeur 1 si il y a un bug et 0 sinon.

In [105]:
data = pd.read_csv('file.csv')
data['Bug'] = data["Fichier"]//10%2
data.head()

,Version,CommitId,Fichier,AvgCyclomatic,AvgCyclomaticModified,Bug
2.1.0,id1,f1,21,41,209,0
2.2.0,id1,f2,91,4,9,1
2.3.0,id2,f1,31,51,19,1
2.4.0,id3,f2,41,41,9,0
2.5.0,id1,f1,21,41,209,0


# Préparation des variables

In [106]:
X = data[['Fichier', 'AvgCyclomatic', 'AvgCyclomaticModified']] # variables indépendantes
y = data['Bug'] # présence d'un bug

# Données d'entraînements et de test

In [107]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42, stratify=y)

# Entrainement du modèle

In [108]:
model1 = LogisticRegression()
model2 = RandomForestClassifier(n_estimators=100, random_state=42)
print(y_train.value_counts())
model1.fit(X_train, y_train)
model2.fit(X_train, y_train)

Bug
1    2
0    2
Name: count, dtype: int64


RandomForestClassifier(random_state=42)

# Prédictions

In [109]:
y_pred1 = model1.predict(X_test)
print("Prédictions logistic regression :", y_pred1)
print("Valeurs réelles :", y_test.values)
y_pred2 = model2.predict(X_test)
print("Prédictions random forest :", y_pred2)
print("Valeurs réelles :", y_test.values)

Prédictions logistic regression : [1 1 0 0]
Valeurs réelles : [0 0 1 1]
Prédictions random forest : [1 1 0 0]
Valeurs réelles : [0 0 1 1]


# Evaluer la performance du modèle

In [110]:
accuracy = accuracy_score(y_test, y_pred1)
print(f"Accuracy logistic regression: {accuracy}")
print(classification_report(y_test, y_pred1))
accuracy = accuracy_score(y_test, y_pred2)
print(f"Accuracy random forest: {accuracy}")
print(classification_report(y_test, y_pred2))

Accuracy logistic regression: 0.0
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       2.0
           1       0.00      0.00      0.00       2.0

    accuracy                           0.00       4.0
   macro avg       0.00      0.00      0.00       4.0
weighted avg       0.00      0.00      0.00       4.0

Accuracy random forest: 0.0
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       2.0
           1       0.00      0.00      0.00       2.0

    accuracy                           0.00       4.0
   macro avg       0.00      0.00      0.00       4.0
weighted avg       0.00      0.00      0.00       4.0



In [111]:
print("Indices de X_train :", X_train.index)
print("Indices de X_test :", X_test.index)

Indices de X_train : Index(['2.7.0', '2.4.0', '2.8.0', '2.3.0'], dtype='object')
Indices de X_test : Index(['2.5.0', '2.1.0', '2.6.0', '2.2.0'], dtype='object')
